In [12]:
import urllib.request
import json
import gzip
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from datasets import Dataset
from datasets import load_dataset, load_from_disk
from accelerate import Accelerator
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer, Seq2SeqTrainingArguments, Trainer, DataCollatorForSeq2Seq, TrainingArguments, TrainerCallback, EarlyStoppingCallback
from torch.utils.data import Dataset
from sympy.parsing.latex import parse_latex
import re
import pandas as pd
import numpy as np
import csv
import sympy as sp
from math import pi, exp
import cmath
from sympy import symbols, sympify, lambdify, Function, Eq, Derivative, integrate, Sum, sqrt, exp, log, sin, cos, tan, cot, sec, csc, pi
from sympy import (symbols, exp, log, sin, cos, tan, csc, sec, cot, 
                   diff, integrate, Sum, prod, Rational, factorial, 
                   expand, factor, trigsimp, I, parse_expr,latex)
import random
import torch
import json
import torch.nn.functional as F
import builtins
import math
from transformers import BitsAndBytesConfig
import ast
import logging

In [13]:
# using cache dir
cache_dir='/workspace/models'

In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base",cache_dir=cache_dir)
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-base",cache_dir=cache_dir)

In [15]:
# Function to extract only the LaTeX expression and Python code
def extract_fields(data):
    return [
        {
            'latex_expression': item['latex_expression'],
            'solution': item['solution'] 
        }
        for item in data
    ]

In [16]:
file_path = '/workspace/SKYLABS PROJECT_dataset/train.json' 
with open(file_path, 'r') as file:
    data = json.load(file)   
data = extract_fields(data)

file_path = 'augmented_equations_amdu2.json'                       #'augmented_equations_amdu.json' #old
with open(file_path, 'r') as file:
    data1 = json.load(file)
data1 = extract_fields(data1)

file_path = 'geometric_amdu2.json'                       #'augmented_equations_amdu.json' #old
with open(file_path, 'r') as file:
    data2 = json.load(file)
data2 = extract_fields(data2)

data = data + data1 + data2

random.shuffle(data)
print(len(data))

127346


In [17]:
#creating training and validation splits
# Determine the split index (e.g., 90% train, 10% validation)
split_index = int(len(data) * 0.9)  # Adjust the ratio as needed (in the skyelectric session they said that keeping this low can avoid out of memory duing backprop. needs testing)

# Split the data
train_data = data[:split_index]
val_data = data[split_index:]

# Print out the first few examples to check
print("Training Data Length:", len(train_data))
print("Validation Data Length:", len(val_data))
# Print out the first few examples to check
print("Training Data Sample:", train_data[:1])
print("Validation Data Sample:", val_data[:1])

Training Data Length: 114611
Validation Data Length: 12735
Training Data Sample: [{'latex_expression': '\\frac{B^{2} R^{2} n^{2} \\left(2 w + 50\\right) \\left(5 y^{3} + 2 z^{3} - 1013 + \\sum_{x=1}^{4} 5^{x}\\right)}{- B R n + 8 \\sin{\\left(2 x \\right)} + 10}', 'solution': 'def augmented_function(B, R, n, w, x, y, z):\n    return B**2*R**2*n**2*(2*w + 50)*(5*y**3 + 2*z**3 - 1013 + Sum(5**x, (x, 1, 4)))/(-B*R*n + 8*sin(2*x) + 10)\n'}]
Validation Data Sample: [{'latex_expression': '1.56470306458227 x^{2} + 0.705482999320612 y^{2} + 1.87319726617967 z + 3.59599259522743 e^{- 1.56603808958583 w}', 'solution': 'from sympy import exp\n\ndef exponential_decay_function(x, y, z, w):\n    return 1.56470306458227*x**2 + 0.705482999320612*y**2 + 1.87319726617967*z + 3.59599259522743*exp(-1.56603808958583*w)\n'}]


In [18]:
#tokenizing
def tokenize_data(data):
    input_text = "Convert LaTeX to Python: " + data['latex_expression']
    target_text = data['solution']
    
    inputs = tokenizer(input_text, truncation=True, padding='max_length', max_length=512, return_tensors=None)#128 was old 256 now
    targets = tokenizer(target_text, truncation=True, padding='max_length', max_length=512, return_tensors=None)
    
    inputs['labels'] = targets['input_ids']
    
    # Replace padding token id's of the labels by -100 so it's ignored by the loss
    inputs['labels'] = [l if l != tokenizer.pad_token_id else -100 for l in inputs['labels']]
    
    return inputs

# Apply tokenization to your data
train_data_tokenized = [tokenize_data(item) for item in train_data]
val_data_tokenized = [tokenize_data(item) for item in val_data]

In [19]:
# Define the Data Collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    return_tensors="pt"
)

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/workspace/saad/project/try3/latex_python_finetune_amdu2",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=val_data_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Fine-tune on Code Search Net Python dataset
trainer.train()

# Save the model after fine-tuning on Code Search Net
model.save_pretrained("/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2")
tokenizer.save_pretrained("/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2")

/opt/conda/envs/llm_bootcamp/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: msaadsalman486 (msaadsalman486-student). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.001600,0.000789
2,0.000800,0.000780
3,0.000700,0.000666
4,0.000500,0.000580


('/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2/tokenizer_config.json',
 '/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2/special_tokens_map.json',
 '/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2/vocab.json',
 '/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2/merges.txt',
 '/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2/added_tokens.json',
 '/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2/tokenizer.json')

In [20]:
# Load the fine-tuned model
cache_dir='/workspace/models'
model = AutoModelForSeq2SeqLM.from_pretrained("/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2",cache_dir=cache_dir).to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/workspace/saad/project/try3/codet5_finetuned_latex_python_amdu2",cache_dir=cache_dir)

In [21]:
# Load test data
with open('/workspace/SKYLABS PROJECT_dataset/public_test_new_no_sol_no_out.json') as f:
    test_data = json.load(f)

In [22]:
sp.ds = sp.dsolve
pi = 3.14159265358979
fresnels = sp.fresnels
fresnelc = sp.fresnelc

# Function to extract the function name and parameters from the generated code
# Define the alphabetic characters from a to z
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z = sp.symbols('a:z')

# Define the additional symbols
x_symb, y_symb, z_symb, d1, d2 = sp.symbols('x_symb y_symb z_symb d1 d2')

def extract_function_name_and_params(code_str):
    match = re.search(r'def\s+(\w+)\s*\((.*?)\):', code_str)
    if match:
        function_name = match.group(1)
        params = [p.strip() for p in match.group(2).split(',') if p.strip()]
        
        sum_matches = re.findall(r'sum\(.+?\*\*(\w+)\s+for\s+\w+\s+in\s+range', code_str)
        for var in sum_matches:
            if var not in params:
                params.append(var)
        
        return function_name, params
    return None, []
    

def evaluate_sum_expressions(code_str):
    sum_pattern = r'Sum\((.*?)\*\*(\w+),\s*\(\2,\s*(\d+),\s*(\d+)\)\)'
    
    def evaluate_sum(match):
        base = match.group(1)
        var = match.group(2)
        start = int(match.group(3))
        end = int(match.group(4))
        result = sum(eval(f"{base}**{i}") for i in range(start, end + 1))
        return str(result)
    
    return re.sub(sum_pattern, evaluate_sum, code_str)

def run_python_code(code_str, test_cases):
    def balance_parentheses(code):
        open_count = code.count('(')
        close_count = code.count(')')
        return open_count, close_count

    function_name, params = extract_function_name_and_params(code_str)
    
    # Extract existing default values
    param_defaults = {}
    for param in params:
        if '=' in param:
            name, default = param.split('=')
            param_defaults[name.strip()] = default.strip()
    
    # Create new parameter string
    new_params = []
    for p in params:
        if '=' in p:
            new_params.append(p)  # Keep existing default values
        else:
            if p in test_cases[0]['input']:
                new_params.append(p)  # Keep as is if in test input
            elif p in param_defaults:
                new_params.append(f"{p}={param_defaults[p]}")  # Use existing default
            else:
                new_params.append(f"{p}=1")  # Add default of 1 if not specified
    
    new_param_str = ', '.join(new_params)
    code_str = re.sub(r'def\s+\w+\s*\(.*?\):', f'def {function_name}({new_param_str}):', code_str)
    
    # Check and fix parentheses
    open_count, close_count = balance_parentheses(code_str)
    
    if open_count > close_count:
        code_str += ')' * (open_count - close_count)
    elif close_count > open_count:
        while close_count > open_count:
            last_close_idx = code_str.rfind(')')
            if last_close_idx != -1:
                code_str = code_str[:last_close_idx] + code_str[last_close_idx + 1:]
                close_count -= 1

    results = []
    try:
        # Evaluate all Sum expressions with powers
        code_str = evaluate_sum_expressions(code_str)
        
        compiled_code = compile(code_str, '<string>', 'exec')
        exec(compiled_code, globals())
    except SyntaxError as e:
        return [f"Error: {e}"] * len(test_cases)
    except Exception as e:
        return [f"Error: {e}"] * len(test_cases)
    
    func = globals().get(function_name)
    
    if func:
        for test_case in test_cases:
            try:
                filtered_input = {k: v for k, v in test_case['input'].items() if k in params}
                result = func(**filtered_input)
                
                if isinstance(result, sp.Expr):
                    result = result.evalf()
                
                if isinstance(result, (int, float, complex, sp.Number)):
                    result = complex(result)
                    result = complex(round(result.real, 6), round(result.imag, 6))
                
                result_str = str(result).replace('I', 'j')
                
                if result_str.endswith('+ 0j') or result_str.endswith('- 0j'):
                    result_str = result_str[:-4]
                
                try:
                    results.append(float(result_str))
                except ValueError:
                    results.append(result_str)
                
            except Exception as e:
                error_message = str(e)
                if "Can't calculate derivative wrt" in error_message:
                    x_val = filtered_input.get('x', None)
                    if x_val is not None:
                        x = sp.Symbol('x')
                        symbolic_result = func(**{k: (x if k == 'x' else v) for k, v in filtered_input.items()})
                        result = symbolic_result.subs(x, x_val).evalf()
                        
                        if isinstance(result, sp.Expr):
                            result = result.evalf()
                        
                        if isinstance(result, (int, float, complex, sp.Number)):
                            result = complex(result)
                            result = complex(round(result.real, 6), round(result.imag, 6))
                        
                        result_str = str(result).replace('I', 'j')
                        #down there 
                        if isinstance(result_str, str) and any(var in result_str for var in ['_symb', 'Symbol']):
                            expr = sp.sympify(result_str)
                            symbols = expr.free_symbols
                            subs_dict = {}
                            for symbol in symbols:
                                symbol_name = str(symbol).replace('_symb', '')
                                if symbol_name in filtered_input:
                                    subs_dict[symbol] = filtered_input[symbol_name]
                            if subs_dict:
                                result = expr.subs(subs_dict).evalf()
                                result_str = str(result)
                        #up there useful or not if not then del
                        if result_str.endswith('+ 0j') or result_str.endswith('- 0j'):
                            result_str = result_str[:-4]
                        
                        try:
                            results.append(float(result_str))
                        except ValueError:
                            results.append(result_str)
                    else:
                        results.append(f"Error: {error_message}")
                else:
                    print(f"Error in test case {test_case}: {e}")
                    results.append(f"Error: {str(e)}")
    else:
        results = ["Function not found"] * len(test_cases)
    
    return results

def process_batch(batch):
    batch_results = []
    for item in batch:
        try:
            input_text = "Convert LaTeX to Python: " + item['latex_expression']
            inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to('cuda')
            
            with torch.no_grad():
                outputs = model.generate(**inputs, max_length=512)
            
            generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print("Generated code:")
            print(generated_code)
            test_cases = item['test_cases']
            
            results = run_python_code(generated_code, test_cases)
            
            formatted_results = []
            for result in results:
                if isinstance(result, float):
                    formatted_results.append(f"{result:.6f}")
                elif isinstance(result, complex):
                    real = round(result.real, 6)
                    imag = round(result.imag, 6)
                    if imag == 0:
                        formatted_results.append(f"{real:.6f}")
                    else:
                        formatted_results.append(f"{real:.6f}{'+' if imag >= 0 else '-'}{abs(imag):.6f}j")
                else:
                    formatted_results.append(str(result))
            
            batch_results.append({
                'task_id': item['task_id'],
                'outputs': formatted_results
            })
        except Exception as e:
            print(f"Error processing item {item['task_id']}: {str(e)}")
            batch_results.append({
                'task_id': item['task_id'],
                'outputs': [f"Error: {str(e)}"] * len(item['test_cases'])
            })
    
    return batch_results

# Prepare results list and set batch size
results_list = []
batch_size = 8  # Adjust based on your GPU memory
# Process each batch of items in the test data
for start_idx in range(0, len(test_data), batch_size):
    batch = test_data[start_idx:start_idx + batch_size]
    batch_results = process_batch(batch)
    results_list.extend(batch_results)
    print(f"Processed batch starting at index {start_idx}")
    
# Save results to CSV
csv_filename = 'results1.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    fieldnames = ['id', 'outputs']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in results_list:
        task_id = result['task_id']
        outputs = result['outputs']
        formatted_outputs = [output if isinstance(output, str) else f"{output:.6f}" for output in outputs]
        writer.writerow({
            'id': task_id,
            'outputs': str(formatted_outputs)  # Convert list to string for CSV
        })
print(f"Results saved to {csv_filename}")

# Load the CSV file
df = pd.read_csv('results1.csv')
# Function to clean and handle errors
def process_output(output):
    # Check if the output is a string and contains specific error messages
    if isinstance(output, str):
        if "Syntax error" in output or "Error" in output:
            # Return a string of zeros with the same number of elements as in the original list
            num_elements = len(eval(output))  # Count the number of elements in the list
            return '[' + ', '.join('0' for _ in range(num_elements)) + ']'
        else:
            # Remove parentheses if it's a valid entry
            return re.sub(r'[()]', '', output)
    return output
# Apply the function to the 'outputs' column
df['outputs'] = df['outputs'].apply(process_output)
# Save the updated DataFrame to a new CSV file
df.to_csv('results2.csv', index=False)
print("Entries processed and saved to 'results2.csv'")

# Read the CSV file
df = pd.read_csv('results2.csv')

# Function to remove '+0j' from the outputs and replace '4*j' with '4j'
def clean_complex_output(output_list):
    # Convert string representation of list to an actual list
    output_list = ast.literal_eval(output_list)
    
    # Function to clean each item
    def clean_item(item):
        item_str = str(item)
        # Remove '+0j'
        item_str = item_str.replace('+0j', '')
        # Replace '4*j' with '4j'
        item_str = re.sub(r'(\d+)\*j', r'\1j', item_str)
        return item_str
    
    # Apply cleaning to each item in the list
    cleaned_list = [clean_item(item) for item in output_list]
    return cleaned_list

# Apply the function to the 'outputs' column
df['outputs'] = df['outputs'].apply(clean_complex_output)

# Save the cleaned DataFrame back to a CSV
df.to_csv('results3.csv', index=False)
print("Done! The cleaned data has been saved to 'results3.csv'.")

Generated code:
def multivariable_function(x, y, z):
    return 6*x**5 + x**4 + 4*x**3 + 2*x**2 + 9*x + 9*y**4 + 9*y**3 + 10*y**2 + 9*y + z**5 + 6*z**4 + 7*z**3 + 8*z**2 + 2*z

Generated code:
def algebraic_function(x):
    return 4*x**6 + 4*x**5 + 3*x**4 + x**3 + 3*x**2 + x + 1

Generated code:
import numpy as np
def geometric_function(r, R):
    return 2513.27384371161*np.pi**2*R*r**2

Generated code:
from sympy import cos

def trigonometric_function(x):
    return cos(4*x) + 8

Generated code:
def diophantine_function(x, y, z, w, v):
    return -6*v - 4*w - 10*x - 2*y + 4*z - 98

Generated code:
import numpy as np
def generated_function(h, B):
    return 17.6772686497683*B*h

Generated code:
def multivariable_function(x, y):
    return 8*x**4 + 3*x**3 + 8*x**2 + 7*x + 3*y**4 + 9*y**3 + 8*y**2 + 7*y + 8

Generated code:

from sympy import symbols, diff, sin, cos, exp, sqrt

# Define the symbols outside of the function
x, a, b, c = symbols('x a b c')

def derivative_by_fun_calling(x_v

<string>:3: RuntimeWarning: invalid value encountered in sqrt


Generated code:
def augmented_function(x, y):
    return (-4*x - 5*y + 92)/(3**x + 10*y - 3844)

Processed batch starting at index 832
Generated code:
def augmented_function(r, w, x, y, z):
    return 7040.08474242056*pi*r**2/(5**x + 5*w**3 + 5*y + 4*z**2 - 64)

Generated code:
def augmented_function(x):
    return (2205.50480367469*log(1629.63367773874*x) + 428.32547462196)*(4*x**5 + x**4 + 2*x**2 + 3*x + 1)

Generated code:
def augmented_function(w, x, y):
    return 8*w - 2*x**5 - 5*x**4 - 5*x**3 - 4*x**2 + 9*x + 6*y**2 - 12 + Sum(9**z, (z, 1, 5))

Generated code:
def augmented_function(x):
    return (x**2/(5*x + 9) - 5.0)*(2*x + 4/(3*x + 5) + 8.42963059682024*exp(-1.93447872093624*x))

Generated code:
def augmented_function(x, y):
    return (4*y**2 - 7225 + Sum(8**x, (x, 1, 4)))/(5*tan(x) + 1)

Generated code:
def augmented_function(x, y):
    return -1.33506067983346*x**2 + (1.08526945963782*x**2 + 6.74931035779982*exp(-1.88513598759289*y))*(8**y + 4*x**3 - 841) - 2.408654286563